# El Lab (DAA Project)

En un laboratorio de la BIOFAM quieren modificar genéticamente una gallina para que sea más resistente a las enfermedades. Para esto deben de cambiar la secuencia de aminoácidos presentes en su genoma. La secuencia que quieren reproducir es la de los cocodrilos, que se sabe que son animales que han existido por millones de años. Entonces, dado que una secuencia de aminoácidos se puede codificar como una secuencia de caracteres se tiene:  

- $T$ es la secuencia de aminoácidos de los cocodrilos (tamaño $m$)  
- $S$ es la secuencia de aminoácidos de la gallina original (tamaño $n$)
- $A$ es la secuencia de aminoácidos de la galllina a crear (inicialmente vacía)

Para lograr que la gallina modificada adquiera la resistencia de un cocodrilo se sabe que su secuencia de aminoácidos $A$, debe contener como prefijo a $T$. Como en la BIOFAM hubo recortes de presupuesto solo cuentan con una máquina que puede hacer dos modificaciones genéticas:  
1. Extrae y elimina el primer aminoácido de $S$ y lo coloca al principio de $A$  
2. Extrae y elimina el primer aminoácido de $S$ y lo coloca al final de $A$  

Encuentre la **cantidad de secuencias de operaciones posibles** tales que la gallina genéticamente modificada sea resistente a las enfermedades.  

Aclaraciones del problema:
+ Se debe modificar la gallina ($A$) con todos los caracteres de $S$, no basta con solo tener el prefijo, se debe llegar al final.

## Soluciones
Para una primera solución se realizó un algoritmo en el que se hace un backtrack por cada una de las operaciones posibles para crear la cadena $S$ comprobando finalmente si contiene a la cadena $T$ como prefijo dando como resultado una secuencia posible de operaciones.

In [30]:
def count_backtrack(T:str, S:str):
    if 0 < len(T) and len(T) <= len(S):
        def backtrack(T:str, S:str, index:int, A:str):
            if len(S) == index:
                if T == A[0:len(T)]:return 1
                return 0
            char = S[index]
            cant = backtrack(T, S, index+1, char+A)
            cant += backtrack(T, S, index+1, A+char)
            return cant
        return backtrack(T, S, 0, "")
    return 0

Probando el algoritmo con distintos casos se obtiene como resultado:

In [31]:
print(count_backtrack('ACAC','ACACBCA'))
print(count_backtrack('ABAB','BABA'))
print(count_backtrack('ACABD','ACABDBCACA'))
print(count_backtrack('DBACA', 'ACABD'))
print(count_backtrack('AABBAA','BABAAA'))
print(count_backtrack('AABBAA','CHAABBAA'))
print(count_backtrack('ACAB','BDACACBA'))

12
4
20
4
12
4
8


Para el ejemplo donde $T$ (prefijo) es "ABAB" y la cadena $S$ es BABA se tiene:  
1. B $\rightarrow$ BA $\rightarrow$ BAB $\rightarrow$ BABA  (poniendo B al principio y al final)  
2. B $\rightarrow$ AB $\rightarrow$ BAB $\rightarrow$ BABA  (poniendo B al principio y al final)  
 
*Resultado*: 4

Como propuesta para una segunda solución se realizó un memoize al algoritmo anterior, ya que en algunos llamados se repetían calculos que ya se habían realizado con anterioridad, por lo que era mejor guardar por subsecuencias de la cadena formada $A$, la subcadena restante de $S$, así como la cantidad de subsecuencias de operaciones posibles realizadas

In [32]:
def count_backtrack_memoize(T:str, S:str):
    if 0 < len(T) and len(T) <= len(S):
        memoize = {}
        def backtrack(T:str, S:str, index:int, A:str):
            if len(S) == index:
                if T == A[0:len(T)]:return 1
                return 0
            char = S[index]
            dict_str = S[index+1:len(S)]
            word_form = char + A
            if dict_str in memoize and word_form in memoize[dict_str]: 
                cant = memoize[dict_str][word_form]
            else: 
                cant = backtrack(T, S, index+1, word_form)
                memoize[dict_str] = {word_form : cant}
            word_form = A + char
            if dict_str in memoize and word_form in memoize[dict_str]: 
                cant += memoize[dict_str][word_form]
            else:
                cant2 = backtrack(T, S, index+1, word_form)
                memoize[dict_str] = {word_form : cant2}
                cant += cant2
            return cant
        return backtrack(T, S, 0, "")
    return 0

Probando el algoritmo se puede comprobar que en efecto se obtienen los mismos resultados:

In [33]:
print(count_backtrack_memoize('ACAC','ACACBCA'))
print(count_backtrack_memoize('ABAB','BABA'))
print(count_backtrack_memoize('ACABD','ACABDBCACA'))
print(count_backtrack_memoize('DBACA', 'ACABD'))
print(count_backtrack_memoize('AABBAA','BABAAA'))
print(count_backtrack_memoize('AABBAA','CHAABBAA'))
print(count_backtrack_memoize('ACAB','BDACACBA'))

12
4
20
4
12
4
8


Se puede ver que aunque ambos algoritmos resuelven el problema, presentan una gran complejidad temporal ya que el backtrack (normal) y el backtrack con memoize (en el peor de los casos no se repiten patrones) pertencen al conjunto $O(2^n)$

Fue necesario encontrar otra vía para solucionar el ejercicio. Partiremos de los posibles casos en que puede aparecer la cadena $T$ (el prefijo), en la cadena creada (genéticamente modificada) $A$.  

Si se comienza a leer la cadena que se desea modificar desde atrás hacia adelante se puede observar que el último elemento puede ser colocado hacia adelante

In [34]:
def count_naive_recursive(T:str, S:str):
    if 0 < len(T) and len(T) <= len(S):
        def naive_recursive(S,T):
            if len(S) < len(T): return 0
            if len(S)==1: 
                if len(T)==1 and T[0]==S[0]: return 2
                return 0
            if len(T)==0 : return 0
            output = naive_recursive(S[0:len(S)-1],T)
            if S[-1]==T[0]: 
                if len(T)==1: output += 2**(len(S)-1)
                else: output += naive_recursive(S[0:len(S)-1],T[1:])
            if len(S)==len(T) and S[-1]==T[-1]: output += naive_recursive(S[0:len(S)-1],T[0:len(T)-1])
            return output
        return naive_recursive(S,T)
    return 0

In [35]:
print(count_naive_recursive('ACAC','ACACBCA'))
print(count_naive_recursive('ABAB','BABA'))
print(count_naive_recursive('ACABD','ACABDBCACA'))
print(count_naive_recursive('DBACA', 'ACABD'))
print(count_naive_recursive('AABBAA','BABAAA'))
print(count_naive_recursive('AABBAA','CHAABBAA'))
print(count_naive_recursive('ACAB','BDACACBA'))

12
4
20
4
12
4
8


In [36]:
def count_dynamic(T:str, S:str):
    if len(T) > len(S): return 0
    dp=[ [ [0 for i in range(len(S))] for i in range(len(T))] for i in range(len(T)) ]
    i=len(T)-1
    while i>=0:
        for j in range(i,len(T)):
            for k in range(len(S)):
                m=j-i
                if m > k:
                    dp[i][j][k]=0
                    continue
                if k==0:
                    if S[0] == T[i]: dp[i][j][k]=2
                    else: dp[i][j][k]=0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
                    continue
                if k >0 : dp[i][j][k]=dp[i][j][k-1]
                if S[k] == T[i]:
                    if m==0: dp[i][j][k]+=2**k
                    elif i<len(T)-1 and k>0: dp[i][j][k]+=dp[i+1][j][k-1]
                if k==m and S[k] == T[j] and j>0 and k>0: dp[i][j][k]+=dp[i][j-1][k-1]
        i-=1
    return dp[0][len(T)-1][len(S)-1]

In [37]:
print(count_dynamic('ACAC','ACACBCA'))
print(count_dynamic('ABAB','BABA'))
print(count_dynamic('ACABD','ACABDBCACA'))
print(count_dynamic('DBACA', 'ACABD'))
print(count_dynamic('AABBAA','BABAAA'))
print(count_dynamic('AABBAA','CHAABBAA'))
print(count_dynamic('ACAB','BDACACBA'))

12
4
20
4
12
4
8


En las siguientes líneas de código se realizó un generador de cadenas que juegan el rol de prefijo (cocodrilo) y genoma a modificar (gallina) haciendo particiones en el prefijo, insertando un caracter random en alguna partición e invirtiendo la secuencia. Se comprueban los resultados entre los algoritmos.

In [42]:
import random
VOC = ["C","T","G","A","U","R","D"]

def generate_prefix(a=10,b=15):
    length = random.randint(a, b)
    prefix = ""
    while length > 0:
        letter = random.randint(0, len(VOC)-1)
        pos = random.randint(0,1)
        if pos:
            prefix += VOC[letter]
        else:
            prefix = VOC[letter] + prefix
        length-=1
    return prefix

def generate_secuence(prefix:str):
    secuence = ""
    while len(secuence) < 40:
        letter = random.randint(0, len(VOC)-1)
        partition = list(prefix.partition(random.choice(prefix)))
        pos = random.randint(0,len(partition))
        partition.insert(pos, VOC[letter])
        secuence += "".join(reversed(partition))
        secuence
    return secuence
   
TEST = 100
while TEST > 0:
    prefix = generate_prefix()
    secuence = generate_secuence(prefix)
    #c_b = count_backtrack(prefix,secuence)
    #c_b_m = count_backtrack_memoize(prefix,secuence)
    c_n_r = count_naive_recursive(prefix,secuence)
    print(len(secuence))
    print(c_n_r)
    c_d = count_dynamic(prefix,secuence)
    #print(c_b, c_b_m, c_n_r, c_d)
    print(c_d)
    print()
    #assert c_b == c_b_m and c_b_m == c_n_r == c_d
    assert c_n_r == c_d
    TEST-=1

42
66
66

48
258
258

48
73728
73728

52
41734
41734

52
2
2

44
72
72

44
384
384

44
13312
13312

45
134
134

42
0
0

48
10
10

48
0
0

45
5356
5356

44
6516
6516

48
370
370

44
453262
453262

48
86
86

45
66
66

42
128
128

52
0
0

44
368
368

48
110486
110486

52
8240
8240

48
1004
1004

42
0
0

45
788
788

45
168
168

48
0
0

42
1302
1302

48
2041508
2041508

42
20
20

52
13416
13416

45
30
30

42
6
6

44
14164
14164

42
188
188

48
168
168

48
4276
4276

45
0
0

44
2856
2856

48
14
14

45
90
90

45
96
96

44
66
66

42
0
0

48
238964
238964

45
42
42

45
22
22

42
560
560

48
32660
32660

48
2088
2088

42
0
0

44
10224
10224

45
0
0

48
624
624

42
162
162

42
52
52

45
0
0

44
11728
11728

52
5384
5384

45
26
26

48
102
102

45
0
0

52
0
0

52
340527908
340527908

52
1120
1120

48
20
20

44
86
86

48
8856
8856

52
258
258

52
417626
417626

45
130
130

42
140880
140880

42
0
0

42
0
0

42
60
60

45
0
0

48
120
120

44
11028
11028

52
13340
13340

42
1824
1824

45
124
124

45
0
0